<a href="https://colab.research.google.com/github/vishesh711/Instagram-Influencers-Prediction/blob/main/Instagram_Influencers_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

UTILS

In [3]:
!pip install openpyxl
!pip install selenium
!pip install bs4
import os, platform, time, urllib.request, openpyxl, operator
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from openpyxl import Workbook
import sys, requests, re, json
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

#Create Empty List
ranking = []
username = []
category = []
category_2 = []

def scrape_username(url):

    #accesing and parsing the input url
    response = requests.get(url)
    print(f'response {response}')
    soup = BeautifulSoup(response.content, 'html.parser')
    list_username = soup.find_all('tr')

    #looping to the element that we want to scrape
    for p in list(list_username):
        try:
            #getting the information (rank, names, and category)
            rank = p.find('td', 'align-middle').get_text().strip()
            ranking.append(rank)
            name = p.find('a').get_text().strip()
            username.append(name)
            cat = p.find_all('span', 'badge badge-pill badge-light samll text-muted')
            category_2 = []
            for c in cat:
                d = c.find('a', 'link').get_text()
                category_2.append(d)
            category.append(category_2)
        except:
            continue

    return ranking, username, category

#function to scrape general information
def scrape_general_info(url):
    # instagram URL
    URL = url

    # creating a dictionary
    data = {}

    # getting the request from url
    r = requests.get(URL)

    # converting the text
    s = BeautifulSoup(r.text, "html.parser")

    # finding meta info
    meta = s.find("meta", property ="og:description")

    #searching followers, followeing and number of posts info
    meta_2 = meta.attrs['content']
    meta_3 = meta_2.split("-")[0].split(" ")

    # assigning the values
    data['Followers'] = meta_3[0]
    data['Following'] = meta_3[2]
    data['Posts'] = meta_3[4]

    return data

#Create Empty List
link = []
names = []

def get_influencer_link(username):
    #to influencer url
    url = f'https://www.instagram.com/{username}/'
    driver = webdriver.Chrome()
    driver.get(url)

    time.sleep(5)

    i = 0
    while i < 8:
        try:
            #get the links
            pages = driver.find_elements_by_tag_name('a')
            for data in pages:
                data_2 = data.get_attribute("href")
                if '/p/' in data_2:
                    link.append(data.get_attribute("href"))
                    names.append(username)
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(1)
            i += 1
        except:
            i += 1
            continue
    driver.quit()

    return link, names


likes = []
comment_counts = []
dates = []
captions = []
type_posts = []
links = []
i = 0
n = 0

def get_information(link):
    try:
        global i, n

        #accessing and parsing the website url
        url = link
        response = requests.get(url)
        soup = BeautifulSoup(response.content)

        #find element that contain information
        body = soup.find('body')
        script = body.find('script')
        raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
        json_data=json.loads(raw)
        posts =json_data['entry_data']['PostPage'][0]['graphql']
        posts= json.dumps(posts)
        posts = json.loads(posts)

        #acquiring information
        like = posts['shortcode_media']['edge_media_preview_like']['count']
        comment_count = posts['shortcode_media']['edge_media_to_parent_comment']['count']
        date = posts['shortcode_media']['taken_at_timestamp']
        caption = posts['shortcode_media']['edge_media_to_caption']['edges'][0]['node']['text']
        type_post = posts['shortcode_media']['__typename']
        likes.append(like)
        comment_counts.append(comment_count)
        dates.append(date)
        captions.append(caption)
        type_posts.append(type_post)
        links.append(link)
        i += 1
    except:
        i += 1
        n += 1
        print(f'number of link error {n} at iteration {i}')
        pass
    return likes, comment_counts, dates, captions, type_posts, links

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00


Scraping Influencer Names

In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def scrape_username(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        Rank = []
        Influencer = []
        category = []

        # Update these selectors based on actual HTML structure
        rows = soup.find_all('div', class_='row')

        for row in rows:
            rank_div = row.find('div', class_='rank')
            if rank_div:
                Rank.append(rank_div.get_text().strip())

            influencer_div = row.find('div', class_='contributor__name-content')
            if influencer_div:
                Influencer.append(influencer_div.get_text().strip())

            category_div = row.find('div', class_='tag__content')
            if category_div:
                category.append(category_div.get_text().strip())

        return Rank, Influencer, category
    else:
        print(f"Failed to retrieve data, status code: {response.status_code}")
        return [], [], []

# Initialize empty lists
username = []
ranking = []
category = []

# Scrape data from multiple pages
for i in range(10):  # Assuming the number of pages is up to 10
    url = f'https://hypeauditor.com/top-instagram/?p={i+1}'
    rank, influencer, cat = scrape_username(url)
    ranking.extend(rank)
    username.extend(influencer)
    category.extend(cat)

# Debugging: Print lengths of lists
print(f"Length of ranking: {len(ranking)}")
print(f"Length of username: {len(username)}")
print(f"Length of category: {len(category)}")

# Check if data was scraped
if not username:
    print("No data was scraped.")
else:
    # Ensure lists are the same length
    min_length = min(len(ranking), len(username), len(category))
    ranking = ranking[:min_length]
    username = username[:min_length]
    category = category[:min_length]

    # Create a dictionary for the data
    inf_dict = {
        'username': username,
        'ranking': ranking,
        'category': category
    }

    # Assign to DataFrame
    df = pd.DataFrame(inf_dict)

    # Do some preprocessing
    df['username_2'] = df['username'].apply(lambda x: x[1:])
    df['category'] = df['category'].astype('str')
    df_2 = df.drop_duplicates().reset_index(drop=True)

    print(inf_dict)

Length of ranking: 500
Length of username: 500
Length of category: 465
{'username': ['cristiano', 'leomessi', 'taylorswift', 'selenagomez', 'kendalljenner', 'billieeilish', 'kyliejenner', 'thv', 'lalalalisa_m', 'arianagrande', 'neymarjr', 'k.mbappe', 'virat.kohli', 'kimkardashian', 'jennierubyjane', 'justinbieber', 'jin', 'mahi7781', 'champagnepapi', 'roses_are_rosie', 'badgalriri', 'sooyaaa__', 'beyonce', 'rkive', 'georginagio', 'kourtneykardash', 'karolg', 'mrbeast', 'anushkasharma', 'katrinakaif', 'narendramodi', 'tomholland2013', 'kingjames', 'rohitsharma45', 'bts.bighitofficial', 'haileybieber', 'eunwo.o_c', 'jlo', 'vinijr', 'mileycyrus', 'judebellingham', 'khaby00', 'realmadrid', 'therock', 'iamcardib', 'ishowspeed', 'dualipa', 'theweeknd', 'gigihadid', 'jennaortega', 'travisscott', 'beingsalmankhan', 'oliviarodrigo', 'fcbarcelona', 'nickiminaj', 'shakira', 'aliaabhatt', 'instagram', 'deepikapadukone', 'adele', 'rashmika_mandanna', 'davidbeckham', 'antonelaroccuzzo', 'virginia', 

In [37]:
df_2.head(500)

,username,ranking,category,username_2
0,cristiano,1,Sports with a ball,ristiano
1,leomessi,2,Sports with a ball,eomessi
2,taylorswift,3,Music,aylorswift
3,selenagomez,4,Photography,elenagomez
4,kendalljenner,5,Modeling,endalljenner
...,...,...,...,...
460,tamaaev,461,Cinema & Actors/actresses,amaaev
461,joaofelix79,462,Sports with a ball,oaofelix79
462,willsmith,463,Cinema & Actors/actresses,illsmith
463,wizkidayo,464,Music,izkidayo


Scraping Influencers Post Links

In [51]:
import os, platform, time, urllib.request, openpyxl, operator
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from openpyxl import Workbook
import sys, requests, re, json
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import utils
from utils import get_influencer_link

In [52]:
inf_names = pd.read_csv(r'influencer_names_final.csv')

In [53]:
inf_names

,username,following,post,followers
0,princessyahrini,649,4769,33500000
1,laudyacynthiabella,911,2588,31500000
2,gisel_la,361,8553,28800000
3,lunamaya,695,3412,27500000
4,natashawilona12,555,854,27100000
...,...,...,...,...
715,persibinfo,30,7643,849100
716,nadyahutagalung,425,8159,855000
717,mike_lewis,916,4168,845500
718,nandishsandhu,714,1070,842800


In [54]:
!sudo apt-get install -y libgbm-dev libx11-xcb1 libxcomposite1 libxrandr2 libxi6 libgconf-2-4 libxss1 libxtst6

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libxcomposite1 is already the newest version (1:0.4.5-1build2).
libxi6 is already the newest version (2:1.8-1build1).
libxrandr2 is already the newest version (2:1.5.2-1build1).
libxss1 is already the newest version (1:1.2.3-1build2).
libxtst6 is already the newest version (2:1.2.3-1build4).
libgconf-2-4 is already the newest version (3.2.6-7ubuntu2).
libgbm-dev is already the newest version (23.2.1-1ubuntu3.1~22.04.2).
libx11-xcb1 is already the newest version (2:1.7.5-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [55]:
#Loopimng for every influencer username
for name in inf_names['username']:
    link, names = get_influencer_link(name)


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /root/.cache/selenium/chrome/linux64/127.0.6533.72/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x5bd69e7666ba <unknown>
#1 0x5bd69e436730 <unknown>
#2 0x5bd69e46e615 <unknown>
#3 0x5bd69e46a488 <unknown>
#4 0x5bd69e4b4e88 <unknown>
#5 0x5bd69e4a87f3 <unknown>
#6 0x5bd69e478ec9 <unknown>
#7 0x5bd69e47991e <unknown>
#8 0x5bd69e72c9eb <unknown>
#9 0x5bd69e730972 <unknown>
#10 0x5bd69e719e15 <unknown>
#11 0x5bd69e731502 <unknown>
#12 0x5bd69e6fed2f <unknown>
#13 0x5bd69e755578 <unknown>
#14 0x5bd69e755750 <unknown>
#15 0x5bd69e76548c <unknown>
#16 0x7b0c265baac3 <unknown>


In [43]:
data_link = pd.DataFrame(link).reset_index(drop=True).rename(columns={
    0:'link'
})

NameError: name 'link' is not defined

In [42]:
data_link['username'] = names

NameError: name 'names' is not defined

In [44]:
data_link_2 = data_link.drop_duplicates().reset_index(drop=True)

NameError: name 'data_link' is not defined

In [56]:
import os
import time
import pandas as pd
import numpy as np
import warnings
from selenium import webdriver

warnings.filterwarnings('ignore')

# Define the Function to Get Influencer Link
def get_influencer_link(username):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run Chrome in headless mode
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    # Provide the correct path to the ChromeDriver executable
    chromedriver_path = '/path/to/your/chromedriver'  # Update this path
    service = webdriver.chrome.service.Service(chromedriver_path)

    driver = webdriver.Chrome(service=service, options=options)
    url = f'https://www.instagram.com/{username}/'
    driver.get(url)

    time.sleep(3)  # Wait for the page to load

    link = driver.current_url
    driver.quit()

    return link, username

# Load Data Influencer Names
inf_names = pd.read_csv('influencer_names_final.csv')


In [59]:
!pip install -U selenium
!python setup.py install

python3: can't open file '/content/setup.py': [Errno 2] No such file or directory


In [61]:
from selenium import webdriver

browser = webdriver.Chrome()
browser.get('http://selenium.dev/')

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /root/.cache/selenium/chrome/linux64/127.0.6533.72/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x5ba98a8516ba <unknown>
#1 0x5ba98a521730 <unknown>
#2 0x5ba98a559615 <unknown>
#3 0x5ba98a555488 <unknown>
#4 0x5ba98a59fe88 <unknown>
#5 0x5ba98a5937f3 <unknown>
#6 0x5ba98a563ec9 <unknown>
#7 0x5ba98a56491e <unknown>
#8 0x5ba98a8179eb <unknown>
#9 0x5ba98a81b972 <unknown>
#10 0x5ba98a804e15 <unknown>
#11 0x5ba98a81c502 <unknown>
#12 0x5ba98a7e9d2f <unknown>
#13 0x5ba98a840578 <unknown>
#14 0x5ba98a840750 <unknown>
#15 0x5ba98a85048c <unknown>
#16 0x7bb0325f5ac3 <unknown>


In [57]:
links = []
usernames = []

for name in inf_names['username']:
    link, username = get_influencer_link(name)
    links.append(link)
    usernames.append(username)

data_link = pd.DataFrame(links, columns=['link'])
data_link['username'] = usernames
data_link_2 = data_link.drop_duplicates().reset_index(drop=True)

print(data_link_2.head())


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
